In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from collections import defaultdict

In [2]:
songs = pd.read_csv('songs.csv')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,customer_id,song_id,score
0,O29219,3459,3
1,I50343,5326,4
2,N42888,236,5
3,F5740,724,4
4,K4115,8452,5


In [4]:
# X_train,X_test = train_test_split(train, test_size = 0.3 , random_state = 0)

In [5]:
# test = X_test
# train = X_train

In [6]:
allRatings = []
data=[]
data1=[]
userRatings = defaultdict(list)

In [7]:
for i in range(len(train)) :
    allRatings.append(train.iloc[i][2])
    userRatings[train.iloc[i][0]].append(train.iloc[i][2])
    data.append(train.iloc[i])

In [8]:
for i in range(len(test)) :
    data1.append(test.iloc[i])

In [10]:
def Predictors(alpha,userB,itemB,i,j,y_u,y_i,userMap,itemMap):
    if(i in userB):
        if(j in itemB):
            rating = alpha+(userB[i])+(itemB[j])
        else:
            rating=alpha+(userB[i])
    else:
        if(j in itemB):
            rating=alpha+itemB[j]
        else:
            rating=alpha
    if i in userMap and j in itemMap:
        rating +=np.inner(y_u[userMap[i]],y_i[itemMap[j]]) 
    return rating   

def Factor_latent_model(lambdabda,tData,vim_Data,factor,trials):
    predictions=[]
    itemTrainDict = defaultdict(lambda: defaultdict(int))
    iValidDict = defaultdict(lambda: defaultdict(int))
    user_train_dict = defaultdict(lambda: defaultdict(int))
    uValidDict = defaultdict(lambda: defaultdict(int))
    userMap = defaultdict(int)
    uCount=0
    itemMap = defaultdict(int)
    userB = defaultdict(float)
    itemB = defaultdict(float)
    iCount=0
    for i in tData:
        user, item, rating = i['customer_id'], i['song_id'], i['score']
        itemTrainDict[item][user] = rating
        user_train_dict[user][item] = rating
        if user not in userMap:
            userMap[user]=uCount
            uCount+=1
        if item not in itemMap:
            itemMap[item]=iCount
            iCount+=1
    
    latentFactor=factor
    np.random.seed(0)
    y_i=np.random.normal(scale=1./latentFactor,size=(len(itemTrainDict),latentFactor ))
    y_u=np.random.normal(scale=1./latentFactor,size=(len(user_train_dict),latentFactor ))

    totalTrials=trials
    alpha = 0
    for counter in range(totalTrials):
        alpha=0
        for i in user_train_dict:
            for j in user_train_dict[i]:
                alpha += user_train_dict[i][j] - userB[i] -itemB[j] - np.inner(y_u[userMap[i]],y_i[itemMap[j]])
        alpha /= len(tData)
        for i in user_train_dict:
            userB[i] = 0
            for j in user_train_dict[i]:
                userB[i] += user_train_dict[i][j]  - alpha - itemB[j] - np.inner(y_u[userMap[i]],y_i[itemMap[j]])
            userB[i] /= (lambdabda + len(user_train_dict[i])) 
        for j in itemTrainDict:
            itemB[j] = 0
            for i in itemTrainDict[j]:
                itemB[j] += itemTrainDict[j][i]  -alpha - userB[i] - np.inner(y_u[userMap[i]],y_i[itemMap[j]])
            itemB[j] /= (lambdabda + len(itemTrainDict[j]))
    
        for i in user_train_dict:
            for lf in range(latentFactor):
                y_u[userMap[i]][lf] = 0
                for j in user_train_dict[i]:
                    y_u[userMap[i]][lf] += y_i[itemMap[j]][lf]*(user_train_dict[i][j]  - alpha - itemB[j]  +y_i[itemMap[j]][lf]*y_i[itemMap[j]][lf]-np.inner(y_u[userMap[i]],y_i[itemMap[j]]) )
                    y_u[userMap[i]][lf]  /= (lambdabda + y_i[itemMap[j]][lf]*y_i[itemMap[j]][lf])
        for j in itemTrainDict:
            for lf in range(latentFactor):
                y_i[itemMap[j]][lf] = 0
                for i in itemTrainDict[j]:
                    y_i[itemMap[j]][lf] += y_u[userMap[i]][lf]*(user_train_dict[i][j]  - alpha - userB[i] - np.inner(y_u[userMap[i]],y_i[itemMap[j]]) +y_u[userMap[i]][lf]*y_u[userMap[i]][lf] )
                    y_i[itemMap[j]][lf] /= (lambdabda + y_u[userMap[i]][lf]*y_u[userMap[i]][lf])
    for i in range(len(vim_Data)) :
        predictions.append(Predictors(alpha,userB,itemB,vim_Data[i][0],vim_Data[i][1],y_u,y_i,userMap,itemMap))
    return MMerror,alpha,userB,itemB,userMap,itemMap,y_u,y_i,predictions

vim_Data = data1
lambdadas=[5.2]
trials=[6]
tData = data
factors=[4]
# factors=[1,2,4,6,8,10,15,20,25,30,35,40,45,50]
bestTrial=1
bestFactor=1
MMerrorList=[]
factorList=[]
lambdadaList=[]
predictions = []
trialList=[]
MMerror = np.iinfo(np.int32).max
bestlambda=0.01
for i in lambdadas:
    tempMMerror=1
    for t in trials:
        for f in factors:
            tempMMerror,alpha,userB,itemB,userMap,itemMap,y_u,y_i,predictions1=Factor_latent_model(i,tData,vim_Data,f,t)
            if(tempMMerror<MMerror):
                bestTrial=t
                bestitemB=itemB
                predictions=predictions1
                bestFactor=f
                bestAlpha=alpha
                bestuserB=userB
                bestlambda=i
                MMerror=tempMMerror
    MMerrorList.append(tempMMerror)
    factorList.append(bestFactor)
    lambdadaList.append(i)

In [11]:
train_predictions = []
train_actual = []
for i in range(len(tData)) :
    train_predictions.append(Predictors(alpha,userB,itemB,tData[i][0],tData[i][1],y_u,y_i,userMap,itemMap))
    train_actual.append(tData[i][2])

In [12]:
mean_squared_error(train_predictions,train_actual)

0.6953862857366426

In [13]:
df1=pd.read_csv("train.csv")
df2=pd.read_csv("songs.csv")
df3=pd.read_csv("save_for_later.csv")
print(df2.shape)
#print(df1)
#df=pd.merge(df1,df2,on="song_id")
df1.head()

(9981, 5)


,customer_id,song_id,score
0,O29219,3459,3
1,I50343,5326,4
2,N42888,236,5
3,F5740,724,4
4,K4115,8452,5


In [14]:
opop=((df3.shape)[0])
print(opop)
df3.head()
save_dict=dict()
for i in range(opop):
    save_dict[(df3.iloc[i][0],df3.iloc[i][1])]=1

238880


In [15]:
print(len(pd.unique(df1["customer_id"])))
print(len(pd.unique(df1["song_id"])))
print(df3.head())
o=pd.unique(df1["customer_id"])
op=pd.unique(df1["song_id"])
u_id=dict()
s_id=dict()
for i in range(len(o)):
    u_id[o[i]]=i
for i in range(len(op)):
    s_id[op[i]]=i
n=len(u_id)
m=len(s_id)
#R=np.zeros((n,m))
#R.shape

14053
10000
  customer_id  song_id
0      F22180     1140
1      N34498     1523
2      K46875     6918
3      L50176     5828
4      F13830     3039


In [16]:
df_labels=pd.read_csv("song_labels.csv")
df_labels=df_labels[df_labels["count"]>0]
print(df_labels.head())
len_label=(df_labels.shape)[0]
z=pd.unique(df_labels["label_id"])
dict_label=dict()
for i in range(len(z)):
    dict_label[z[i]]=i

  platform_id  label_id  count
0     Q207781     10304      6
1     S961833     29011     15
2    X6718608     25148     29
3   Q16134551      8717    774
4       V7126     15169    120


In [17]:
songs_alphan=dict()
op=(df2.shape)[0]
for i in range(op):
    songs_alphan[df2.iloc[i][1]]=df2.iloc[i][0]
print(songs_alphan["W9597"])

2552


In [18]:
useful=[]
for i in z:
    useful.append((len(df_labels[df_labels["label_id"]==i]),i))
useful.sort()
useful.reverse()

In [19]:
useful=useful[:400]

In [20]:
vector_rep=dict()
op=(df_labels.shape)[0]
print(op)
for i in songs_alphan:
    vector_rep[songs_alphan[i]]=[0 for i in range(400)]

999906


In [21]:
feature_map=dict()
for i in range(400):
    feature_map[useful[i][1]]=i

In [22]:
print(feature_map[22743])

2


In [23]:
for i in range(op):
    if(df_labels.iloc[i][1] not in feature_map):
        continue
    if(df_labels.iloc[i][0] not in songs_alphan):
        continue
    if(1+df_labels.iloc[i][2]==0):
        print("OK")
    vector_rep[songs_alphan[df_labels.iloc[i][0]]][feature_map[df_labels.iloc[i][1]]]+=(df_labels.iloc[i][2])

In [24]:
print(songs_alphan["Q16134551"])
print(vector_rep[4185])
#print(feature_map[10304])

4185
[1928, 897, 26, 39, 774, 0, 0, 27, 0, 213, 0, 49, 0, 14, 57, 0, 0, 0, 16, 15, 14, 77, 0, 0, 197, 20, 236, 25, 20, 33, 16, 0, 20, 33, 0, 0, 0, 541, 0, 0, 0, 25, 0, 0, 0, 0, 0, 0, 303, 0, 0, 43, 29, 0, 40, 0, 0, 0, 178, 26, 0, 0, 24, 0, 18, 74, 0, 0, 17, 48, 0, 0, 0, 0, 0, 0, 0, 19, 25, 0, 25, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 20, 0, 0, 0, 19, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 17, 0, 0, 30, 0, 0, 0, 0, 30, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 21, 0, 0, 0, 0, 0, 0, 17, 0, 0, 19, 0, 0, 0, 0, 0, 118, 203, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 22, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 464, 0, 0, 0, 0, 80, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
print(songs_alphan["P10"])
print(vector_rep[3753][feature_map[17124]])
o=0
for i in vector_rep[3753]:
    o=o+(i*i)
print(o)
#print(feature_map[10304])

3753
6
10615471


In [26]:
user_vec=dict()
count_vec=dict()
for i in u_id:
    user_vec[i]=[0 for i in range(400)]
    count_vec[i]=0
print(len(vector_rep))

9981


In [27]:
bbop=pd.read_csv("save_for_later.csv")
for i in range((bbop.shape)[0]):
    user=bbop.iloc[i][0]
    item=bbop.iloc[i][1]
    if(item in vector_rep):
        pp=vector_rep[item]
        pp=pp[:400]
    else:
        pp=[0 for ioio in range(400)]
    for ioio in range(400):
        user_vec[user][ioio]+=(pp[ioio])
bbop=train
for i in range((bbop.shape)[0]):
    if(bbop.iloc[i][2]==3 or bbop.iloc[i][2]==4):
        continue
    if(bbop.iloc[i][2]==5):
        user=bbop.iloc[i][0]
        item=bbop.iloc[i][1]
        if(item in vector_rep):
            pp=vector_rep[item]
            pp=pp[:400]
        else:
            pp=[0 for ioio in range(400)]
        for ioio in range(400):
            user_vec[user][ioio]+=(pp[ioio])
    else:
        user=bbop.iloc[i][0]
        item=bbop.iloc[i][1]
        if(item in vector_rep):
            pp=vector_rep[item]
            pp=pp[:400]
        else:
            pp=[0 for ioio in range(400)]
        for ioio in range(400):
            user_vec[user][ioio]-=(pp[ioio])

In [28]:
for i in vector_rep:
    sumi=0
    for j in vector_rep[i]:
        sumi+=(j*j)
    sumi=np.sqrt(sumi)
    for j in range(400):
        vector_rep[i][j]=vector_rep[i][j]/sumi

In [29]:

for i in user_vec:
    sumi=0
    for j in user_vec[i]:
        sumi+=(j*j)
    if(sumi==0):
        continue
    sumi=np.sqrt(sumi)
    for j in range(400):
        user_vec[i][j]=user_vec[i][j]/sumi
print(user_vec["J16874"][feature_map[25148]])
pok=0
for i in user_vec["J16874"]:
    pok=pok+(i*i)
print(pok)

0.0005218086422891419
0.9999999999999984


In [30]:
mean_op=[0 for i in range(400)]
for i in vector_rep:
    for j in range(400):
        mean_op[j]+=vector_rep[i][j]
z=0
for i in range(400):
    z=z+(mean_op[i])
for i in range(400):
    mean_op[i]=mean_op[i]/z

In [31]:
x_train_newer=[]
y_train_newer=[]
pl=0
for i in range((train.shape[0])):
    x_train_newer.append([])
    user=train.iloc[i][0]
    zz=user_vec[user]
    item=train.iloc[i][1]
    for pp in zz:
        x_train_newer[pl].append(pp)
    if(item in vector_rep):
        zz=vector_rep[item]
        zz=zz[:400]
    else:
        zz=mean_op
    for pp in zz:
        x_train_newer[pl].append(pp)
    x_train_newer[pl].append(train_predictions[pl])
    y_train_newer.append(train.iloc[i][2])
    pl+=1
print(x_train_newer[0])

[-0.9700534690868696, -0.12384263821228908, -0.029146143590797962, -0.033901745222203905, -0.14757755353900184, -0.007399654577281476, -0.010129893183790907, -0.06817901368002692, -0.007242673552555455, -0.008572395173764107, -0.004872567885123368, -0.005906795812730097, -0.005866781041721504, -0.006306943522816034, -0.004503200768120965, -0.002188500168239239, -0.004284658557227877, -0.004934129071290436, -0.0047125088010889936, -0.004570918072904739, -0.004023023516017841, -0.015113271204014997, -0.005229622764892358, -0.002379339845357147, -0.0015328735355599732, -0.003013420062877939, -0.01474082602770424, -0.0019391773642626166, -0.0017329473906029416, -0.005737502550770663, -0.002668677420342363, -0.0029457027580941653, -0.0018622258815537826, -0.004084584702184908, -0.010967125315663021, -0.0020499874993633376, -0.027533240513220803, -0.01991504372504624, -0.0009972912159064886, -0.001729869331294588, -0.0020776900331385176, -0.004084584702184908, -0.0003355084646105162, -0.0005

In [32]:
print(len(x_train_newer))

710109


In [33]:
x_test_newer=[]
pl=0
for i in range((test.shape[0])):
    x_test_newer.append([])
    user=test.iloc[i][0]
    item=test.iloc[i][1]
    zz=user_vec[user]
    for pp in zz:
        x_test_newer[pl].append(pp)
    if(item in vector_rep):
        zz=vector_rep[item]
        zz=zz[:400]
    else:
        zz=mean_op
    for pp in zz:
        x_test_newer[pl].append(pp)
    x_test_newer[pl].append(Predictors(alpha,userB,itemB,test.iloc[i][0],test.iloc[i][1],y_u,y_i,userMap,itemMap))
    pl+=1
print(x_test_newer[0])

[0.9976273253037533, 0.0206602023076809, 0.004803319802983921, 0.004044411544733618, 0.05661013975401567, 0.0013121093775874295, 0.001531762763206583, 0.024601179189345183, 0.0012040259656161002, 0.0016944689747763263, 0.0014085709173037775, 0.0007809898155347677, 0.00081120668339772, 0.0015910343117069895, 0.0005473901832096363, 0.0007403132626423319, 0.0009169657209180531, 0.0014283281001372461, 0.0005543633065626252, 0.001477139963608169, 0.0009773994566439578, 0.0015852233755794987, 0.0014248415384607517, 0.00039049490776738385, 0.0007100963947793796, 0.00038700834609088934, 0.00024289713012911674, 4.648748901992665e-05, 0.0003939814694438783, 0.0002986821169530287, 0.0001998962027856846, 0.0010215625712128881, 4.997405069642115e-05, 3.835217844143948e-05, 0.0024359444246441564, 0.0008228285556527016, 0.003405208570709627, 0.00025219462793310207, 0.0004974161325132151, 0.0008658294829961338, 0.0008146932450742145, 0.0006368785995729951, 0.0001720037093737286, 0.0002812493085705562,

In [34]:
# y_test_newer=[]
# for i in range((test.shape)[0]):
#     rr=test.iloc[i][2]
#     y_test_newer.append(rr)

In [35]:
print(x_test_newer[0])

[0.9976273253037533, 0.0206602023076809, 0.004803319802983921, 0.004044411544733618, 0.05661013975401567, 0.0013121093775874295, 0.001531762763206583, 0.024601179189345183, 0.0012040259656161002, 0.0016944689747763263, 0.0014085709173037775, 0.0007809898155347677, 0.00081120668339772, 0.0015910343117069895, 0.0005473901832096363, 0.0007403132626423319, 0.0009169657209180531, 0.0014283281001372461, 0.0005543633065626252, 0.001477139963608169, 0.0009773994566439578, 0.0015852233755794987, 0.0014248415384607517, 0.00039049490776738385, 0.0007100963947793796, 0.00038700834609088934, 0.00024289713012911674, 4.648748901992665e-05, 0.0003939814694438783, 0.0002986821169530287, 0.0001998962027856846, 0.0010215625712128881, 4.997405069642115e-05, 3.835217844143948e-05, 0.0024359444246441564, 0.0008228285556527016, 0.003405208570709627, 0.00025219462793310207, 0.0004974161325132151, 0.0008658294829961338, 0.0008146932450742145, 0.0006368785995729951, 0.0001720037093737286, 0.0002812493085705562,

In [36]:
# x_train_newer = pd.read_csv('x_train_newer.csv')
# y_train_newer = pd.read_csv('y_train_newer.csv')

In [37]:
abc = pd.DataFrame(x_train_newer)
abc.to_csv('x_train.csv',index=False)

In [38]:
abc = pd.DataFrame(x_test_newer)
abc.to_csv('x_test.csv',index=False)

In [39]:
abc = pd.DataFrame(y_train_newer)
abc.to_csv('y_train.csv',index=False)

In [40]:
# x_test_newer = pd.read_csv('x_test_newer.csv')
# y_test_newer = pd.read_csv('y_test_newer.csv')

In [ ]:
from xgboost import XGBClassifier
model_new=XGBClassifier()
model_new.fit(np.array(x_train_newer),np.array(y_train_newer))

C:\Users\sanch\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [ ]:
predictions_ff_new=model_new.predict(np.array(x_test_newer))

In [ ]:
# print(mean_squared_error(predictions_ff_new,y_test_newer))

In [ ]:
print(predictions_ff_new[0])

In [ ]:
pred_proba=model_new.predict_proba(np.array((x_test_newer)))
print(pred_proba[0])

In [ ]:
zz=[]
for i in pred_proba:
    zz.append((i[0]*1)+(i[1]*2)+(i[2]*3)+(i[3]*4)+(i[4]*5))
print(mean_squared_error(zz,predictions_ff_new))

In [ ]:
# print(mean_squared_error(zz,y_test_newer))

In [ ]:
from xgboost import XGBRegressor
model_new_new=XGBRegressor()
model_new_new.fit(np.array(x_train_newer),np.array(y_train_newer))

In [ ]:
predictions_new_new=model_new_new.predict(np.array(x_test_newer))

In [ ]:
# print(mean_squared_error(predictions_new_new,y_test_newer))

In [ ]:
from lightgbm import LGBMRegressor,LGBMClassifier
model=LGBMRegressor()
model.fit(x_train_newer,y_train_newer)
predictions_ff=model.predict(x_test_newer)

In [ ]:
model_newest=LGBMClassifier()
model_newest.fit(x_train_newer,y_train_newer)
predictions_newest=model_newest.predict(x_test_newer)

In [ ]:
cc=model_newest.predict_proba(x_test_newer)

In [ ]:
lol=[]
for i in cc:
    lol.append((i[0])+(2*i[1])+(3*i[2])+(4*i[3])+(5*i[4]))

In [ ]:
aa=[]
for i in range(len(lol)):
    aa.append((3*zz[i]+lol[i]+predictions_ff[i]+2*predictions_new_new[i])/7)

In [ ]:
# mean_squared_error(aa,y_test_newer)

In [ ]:
df_write = pd.DataFrame(data=aa , columns=['score'])
df_write['test_row_id'] = np.arange(len(df_write))
df_write = df_write[['test_row_id','score']]
df_write.to_csv('Answer_v20.csv',index=False)